In [ ]:
import json
import pandas as pd
import numpy as np
from joblib import dump
import requests

from sqlalchemy import create_engine
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing

## Build Model and Serialize (use pick to save to disk)
Note: in real life you'd also have some feature engineering/preprocessing functions that you'd use for both training the model and predicting inside the Rest API You'd also need to serialize any sklearn preprocessing objects.

In [ ]:
# load dataset from db
db = create_engine('sqlite:///data/data.sqlite')
df = pd.read_sql_table('iris', db)

# create feature and target dfs
features = [
    'sepal_length',
    'sepal_width',
    'petal_length',
    'petal_width'
]
target = 'species'

X = df[features]
y = df[target]

# fit model
rfc = RandomForestClassifier(random_state=42)
rfc.fit(X, y)

# serialize
dump(rfc, 'iris_rfc.joblib')

## Test Model using Rest API
Note: get the api running first with: `uvicorn dope_model_api:app --host 127.0.0.1 --port 8000 --reload `

In [ ]:
df = pd.DataFrame({'sepal_length': [0, 1], 'sepal_width': [1, 2], 'petal_length': [2, 3], 'petal_width': [3, 4]})
body = {'data': df.to_dict(orient='records')}

r = requests.post('http://localhost:8000/predict/', data=json.dumps(body))
print(r.json())